In [1]:
import os
import websockets
import json
import pandas as pd
import asyncio
import ta
import numpy as np
from binance.client import Client
import time
from datetime import datetime, timedelta

In [2]:
# User Input
COIN_NAME = 'COCOS'
TRADE_SIZE_USD = 10
GRADIENT = 600

In [3]:
# Initialise env variables
API_KEY = os.environ.get('BINANCE_API_KEY')
API_SECRET = os.environ.get('BINANCE_API_SECRET')

# Initialise Binance Client
client = Client(API_KEY, API_SECRET)

# Init buy / sell vars
COIN_PAIR = COIN_NAME + 'USDT'
BUY_AMOUNT_USD = float(TRADE_SIZE_USD)

# Init vars
LONG_PAUSE = 30
SHORT_PAUSE = 5

# Init stop loss / take profit
STOP_PROFIT = 1.005
STOP_LOSS = 0.995

# Set display for all rows
pd.set_option('display.max_rows', None)

COIN_PRICE = float(client.get_symbol_ticker(symbol=COIN_PAIR)['price'])
print(f'Price of {COIN_PAIR}: {COIN_PRICE} {COIN_PAIR[3:-1]}')
BUY_AMOUNT_COIN = round(BUY_AMOUNT_USD / COIN_PRICE, 8)
print(f'Amount of {COIN_PAIR} to buy: {BUY_AMOUNT_COIN} {COIN_PAIR[0:-4]}')

Price of COCOSUSDT: 2.669 OSUSD
Amount of COCOSUSDT to buy: 3.74672162 COCOS


In [4]:
def getMinuteData(symbol = COIN_PAIR, interval = '1m', minutes = '60'):
    # startTime = datetime.utcnow() - timedelta(minutes=minutes)
    # startTime = datetime.utcnow() - timedelta(minutes=60)
    frame = pd.DataFrame(client.get_historical_klines(symbol, interval, minutes + ' min ago UTC'))
    # frame = pd.DataFrame(client.get_historical_klines(symbol, interval, startTime))
    frame = frame.iloc[:,:6]
    frame.columns = ['Time', 'Open', 'High', 'Low', 'Close', 'Volume']
    frame = frame.set_index('Time')
    frame.index = pd.to_datetime(frame.index, unit = 'ms')
    frame = frame.astype(float)
    return frame

In [15]:
df5 = getMinuteData(symbol = COIN_PAIR, interval = '5m', minutes = '540')
df1 = getMinuteData(symbol = COIN_PAIR, interval = '1m', minutes = '540')

In [16]:
def applyTechnicals(df, interval = 1):
    df['%K'] = ta.momentum.stoch(df.High, df.Low, df.Close, window = 14, smooth_window = 3)
    df['%D'] = df['%K'].rolling(3).mean()
    df['RSI'] = ta.momentum.rsi(df.Close, window = 14)
    # df['MACD'] = ta.trend.macd(df.Close)
    # df['MACD-D'] = ta.trend.macd_diff(df.Close)
    # df['MACD-S'] = ta.trend.macd_signal(df.Close)
    gradientFactor = (GRADIENT) / interval
    df['SMA-5'] = ta.trend.sma_indicator(df.Close, window = 5)
    df['SMA-8'] = ta.trend.sma_indicator(df.Close, window = 8)
    df['SMA-13'] = ta.trend.sma_indicator(df.Close, window = 13)
    df['SMA-5-M'] = (df['SMA-5'] - df['SMA-5'].shift(1))*gradientFactor
    df['SMA-8-M'] = (df['SMA-8'] - df['SMA-8'].shift(1))*gradientFactor
    df['SMA-13-M'] = (df['SMA-13'] - df['SMA-13'].shift(1))*gradientFactor
    # df.dropna(inplace = True)

In [17]:
applyTechnicals(df1, 1)
applyTechnicals(df5, 5)

In [45]:
def strategy(open_position = False):
    profitTrade = 0.0
    profitTotal = 0.0
    buyPrice = 0.0
    sellPrice = 0.0
    
    while True:
        print("Waiting for 5m buy signal...")
        
        df5 = getMinuteData(symbol = COIN_PAIR, interval = '5m', minutes = '360')
        applyTechnicals(df5, 5)

        # Look for trigger on 5 min chart to buy
        if df5['SMA-5-M'][-1] > -1:
            
            if not open_position:
                print("Waiting for 1m buy signal...")
                while True:
                    df1 = getMinuteData(symbol = COIN_PAIR, interval = '1m', minutes = '30')
                    applyTechnicals(df1, 1)

                    # Look for trigger on 1 min chart to buy
                    if df1['SMA-5-M'][-1] > -1.5:
                        # Buy Signal
                        open_position = True
                        buyPrice = df1['Close'][-1]
                        print(f"BUY @ {df1.index[-1]} - {buyPrice}")
                        break

                    time.sleep(SHORT_PAUSE)
            
            if open_position:
                print("Waiting for 5m sell signal...")
                while True:
                    df5 = getMinuteData(symbol = COIN_PAIR, interval = '5m', minutes = '180')
                    applyTechnicals(df5, 5)
                    
                    # Look for trigger on 5 min chart to sell
                    if df5['SMA-5-M'][-1] < 3:
                        print("Waiting for 1m sell signal...")
                        while True:
                            df1 = getMinuteData(symbol = COIN_PAIR, interval = '1m', minutes = '30')
                            applyTechnicals(df1, 1)

                            # Look for trigger on 1 min chart to sell
                            if df1['SMA-5-M'][-1] < 0:
                                # Sell Signal
                                open_position = False
                                sellPrice = df1['Close'][-1]
                                profitTrade =  sellPrice - buyPrice
                                profitTotal += profitTrade
                                print(f"SELL @ {df1.index[-1]} - {sellPrice}")
                                print("--------")
                                print(f"Profit Trade: {profitTrade}")
                                print(f"Profit Total: {profitTotal}")
                                print("--------")
                                break

                            time.sleep(SHORT_PAUSE)
                        
                    if not open_position:
                        break
                        
                    time.sleep(LONG_PAUSE)
        
        time.sleep(LONG_PAUSE)
            
    

In [ ]:
strategy()

In [18]:
df1

,Open,High,Low,Close,Volume,%K,%D,RSI,SMA-5,SMA-8,SMA-13,SMA-5-M,SMA-8-M,SMA-13-M
Time,,,,,,,,,,,,,,
2021-11-22 11:40:00,3.4988,3.5374,3.4250,3.4552,333274.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-11-22 11:41:00,3.4535,3.4699,3.4262,3.4611,99016.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-11-22 11:42:00,3.4604,3.4770,3.4251,3.4333,177231.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-11-22 11:43:00,3.4293,3.4435,3.3820,3.4310,131578.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-11-22 11:44:00,3.4253,3.4322,3.3889,3.4011,97330.0,NaN,NaN,NaN,3.43634,NaN,NaN,NaN,NaN,NaN
2021-11-22 11:45:00,3.4053,3.4187,3.3941,3.4149,46373.0,NaN,NaN,NaN,3.42828,NaN,NaN,-4.836,NaN,NaN
2021-11-22 11:46:00,3.4181,3.4200,3.4014,3.4014,53218.0,NaN,NaN,NaN,3.41634,NaN,NaN,-7.164,NaN,NaN
2021-11-22 11:47:00,3.4025,3.4025,3.3697,3.3741,79638.0,NaN,NaN,NaN,3.40450,3.421512,NaN,-7.104,NaN,NaN
2021-11-22 11:48:00,3.3761,3.3761,3.3049,3.3321,168913.0,NaN,NaN,NaN,3.38472,3.406125,NaN,-11.868,-9.2325,NaN


In [10]:
df5

,Open,High,Low,Close,Volume,RSI,SMA-5,SMA-8,SMA-13,SMA-5-M,SMA-8-M,SMA-13-M
Time,,,,,,,,,,,,
2021-11-22 14:40:00,3.0368,3.1000,3.0368,3.0904,70586.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-11-22 14:45:00,3.0904,3.1255,3.0801,3.1170,155550.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-11-22 14:50:00,3.1199,3.1240,3.0800,3.1118,133530.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-11-22 14:55:00,3.1071,3.1109,3.0600,3.0647,88464.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-11-22 15:00:00,3.0686,3.0686,2.9909,3.0360,418446.0,NaN,3.08398,NaN,NaN,NaN,NaN,NaN
2021-11-22 15:05:00,3.0360,3.0469,2.9877,3.0240,113220.0,NaN,3.07070,NaN,NaN,-1.5936,NaN,NaN
2021-11-22 15:10:00,3.0251,3.0564,3.0199,3.0311,63320.0,NaN,3.05352,NaN,NaN,-2.0616,NaN,NaN
2021-11-22 15:15:00,3.0312,3.0356,3.0001,3.0047,80992.0,NaN,3.03210,3.059963,NaN,-2.5704,NaN,NaN
2021-11-22 15:20:00,3.0047,3.0292,2.9800,2.9881,160847.0,NaN,3.01678,3.047175,NaN,-1.8384,-1.5345,NaN
